<a href="https://colab.research.google.com/github/alexisakov/NLPTutorial/blob/main/NLPExcercise01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-08-12 05:06:42--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.242, 5.45.205.244, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.242|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-08-12 05:06:42--  http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)... 5.45.220.18, 2a02:6b8:0:2002::19
Connecting to cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)|5.45.220.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [39]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [40]:
from pymystem3 import Mystem
tqdm.pandas() 

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [29]:
df_train = pd.read_csv('https://github.com/alexisakov/NLPTutorial/raw/main/train.csv.zip'
  , sep=';')
df_test = pd.read_csv('https://github.com/alexisakov/NLPTutorial/raw/main/test.csv.zip'
  , sep=';')

In [31]:
df_train['text'] = df_train['text'].progress_map(lambda x: BeautifulSoup(x, 'html.parser').text)

In [ ]:
df_train.head()

In [33]:
mystm = Mystem()

In [42]:
df_train['text'] = df_train['text'].progress_map(lambda x: ''.join(mystm.lemmatize(x)).rstrip() )

In [37]:
df_train.head()

,id,text,target
0,0,в молодой и дружный коллектив динамично развив...,0
1,1,обязанность: - прием входить звонок и заявка;-...,0
2,2,федеральный сеть аптека в связь с активный рас...,0
3,3,обязанность: разработка проектный и рабочий ...,0
4,4,производитель сантехника тм Domani Spa требова...,1


In [43]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)

In [45]:
tfidf_vectorizer.fit(df_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [46]:
X = tfidf_vectorizer.transform(df_train['text'])

In [48]:
Y = df_train['target']

In [58]:
log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0)

In [62]:
parameters = {
     'C': [0.1,0.5,1.0,2.0,5.0], 'penalty': ['l2','l1']}
clf = GridSearchCV(log_reg, parameters,
                   scoring = 'roc_auc',
                   cv = 5)

In [64]:
clf.fit(X,Y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=1,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.5, 1.0, 2.0, 5.0],
                         'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [65]:
clf.best_params_

{'C': 5.0, 'penalty': 'l2'}

In [66]:
parameters = {
     'C': [5.0,10,15,50,100]}
clf = GridSearchCV(log_reg, parameters,
                   scoring = 'roc_auc',
                   cv = 5)

In [67]:
clf.fit(X,Y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=1,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [5.0, 10, 15, 50, 100]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='roc_auc',
             verbose=0)

In [68]:
clf.best_params_

{'C': 5.0}

In [69]:
parameters = {
     'C': [5.0, 6.0, 7.0, 8.0], 'penalty': ['l2','l1']}
clf = GridSearchCV(log_reg, parameters,
                   scoring = 'roc_auc',
                   cv = 5)

In [70]:
clf.fit(X,Y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=1,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [5.0, 6.0, 7.0, 8.0], 'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [71]:
clf.best_params_

{'C': 5.0, 'penalty': 'l2'}

# Apply the model to test data

In [72]:
df_test['text'] = df_test['text'].progress_map(lambda x: BeautifulSoup(x, 'html.parser').text)

In [73]:
df_test['text'] = df_test['text'].progress_map(lambda x: ''.join(mystm.lemmatize(x)).rstrip() )

In [74]:
X_test = tfidf_vectorizer.transform(df_test['text'])

In [78]:
df_test.head()

,id,text
0,31063,в крупный компания по организация и приготовле...
1,31064,обязанность: обеспечение необходимый функцион...
2,31065,обязанность: отгрузка и прием товар со скла...
3,31066,обязанность: приготовление холодный и горячий...
4,31067,вакансия срочный! внимание! просьба подробно и...


In [82]:
df_test[['target']] = clf.best_estimator_.predict_proba(X_test)[:,1]

In [83]:
df_test.head()

,id,text,target
0,31063,в крупный компания по организация и приготовле...,0.999758
1,31064,обязанность: обеспечение необходимый функцион...,0.004606
2,31065,обязанность: отгрузка и прием товар со скла...,0.998496
3,31066,обязанность: приготовление холодный и горячий...,0.990171
4,31067,вакансия срочный! внимание! просьба подробно и...,0.923102


In [85]:
clf.best_estimator_.predict(X_test)

array([1, 0, 1, ..., 0, 1, 1])

In [86]:
from google.colab import files

In [89]:
df_test[{'id','target'}].head()

,id,target
0,31063,0.999758
1,31064,0.004606
2,31065,0.998496
3,31066,0.990171
4,31067,0.923102


In [91]:
df_test[{'id','target'}].to_csv('scoredtest.csv',index=False) 
files.download('scoredtest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>